# A few quick things

* [Can you execute only the selected text in a notebook cell?](https://github.com/jupyter/notebook/issues/252)
    * Yes, but you have to do [a bit of work](https://carreau.gitbooks.io/jupyter-book/content/custom-extension-1.html)
* Useful tip: you can get help with ?
    * e.g., `?np.random.normal`
* Useful tip 2: you can execute shell commands with !
    * e.g., `!conda install seaborn`

# Overview of Day 3
* Preliminaries
* Describing and summarizing data
* Visualization with matplotlib and seaborn
* Statistical analysis in SciPy and statsmodels
* Bayesian modeling in PyMC3

# Preliminaries
* Import stuff
* Load the preprocessed data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

%matplotlib inline

# Disable annoying SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [ ]:
data = pd.read_csv('../data/preprocessed_data.csv')

# Describing and summarizing data
* It's a good idea to explore your data before you start modeling
* Surprising how often interesting results reflect overlooked problems

### A bird's eye view
* Simple pandas tools for describing/inspecting entire datasets
    * .describe(), .head() and .tail(), .dtypes(), etc...

In [ ]:
# Describe the continuous variables in the dataset
# for both categorical and continuous variables, with quartiles
data.describe(percentiles=[0.25, 0.5, 0.75])

## Frequency counts
* 1-dimensional frequency counts can provide quick insight, diagnose problems
* What are the [top dog and cat names](http://dogtime.com/dog-names/female-dog-names/21032-top-10-most-popular-dog-and-cat-names-of-2014) in Austin?

In [ ]:
top_n = 20
for species in ['Dog', 'Cat']:
    name_counts = data.query('animal == @species')['name'].value_counts()
    print("\nTop {} {} names:\n{}".format(top_n, species, name_counts[:top_n]))

### What the heck kind of name is "X"?

In [ ]:
data.query('name == "X"').head(10)

### Contingency tables
* It's useful to examine contingency tables (or crosstabs) for pairs of variables
* Normalized crosstabs can be particularly informative

In [ ]:
# Select only dogs
dogs = data.query('animal=="Dog"')

# Keep only common breeds--estimates won't be reliable for uncommon ones
dogs = dogs.groupby('breed').filter(lambda x: len(x) >= 100)

# How many breeds left?
print("Unique breeds:", dogs['breed'].nunique())

# Generate contingency table
ctab = pd.crosstab(dogs['breed'], data['outcome'])

# Sort in descending order of number of animals of each breed
ctab = ctab.loc[ctab.sum(1).sort_values(ascending=False).index]

# Show just the top 10
ctab[:10]

#### Normalized crosstabs
We can normalize the crosstab by dividing by the sum of the rows and/or columns

In [ ]:
# Normalize by row to show proportion of animals accounted for by each outcome
nctab = ctab.divide(ctab.sum(1), 0).round(2)
nctab[:10]

#### Drilling down further
* Euthanasia is a pretty general category
* Why are pit bulls being euthanized at such a high rate?
* Repeat with outcome subtype

In [ ]:
# We can do the same thing for outcome subtype
sctab = pd.crosstab(dogs['breed'], data['outcome_subtype'])
sctab = sctab.divide(sctab.sum(1), 0).round(2)

# We could add a column for number of dogs in each breed
# sctab['number'] = dogs.groupby('breed')['animal'].count()

# sort in descending order of 'aggressive'
sctab.sort_values('Aggressive', ascending=False)[:10]

### What does this mean?
1 in 3 pit bulls that pass through AAC are euthanized! Some possible interpretations:
* Pit bulls are much more dangerous than other dog breeds
    * Could be genetic
    * Could be abusive owners
* There could be a bias against pit bulls, causing many to be unnecessarily euthanized
* Mixed-breed dogs with behavioral problems may be more likely to be labeled pit bulls
* Others?

# Visualization in Python
* That thing about a picture being worth...
* Python has a rich visualization ecosystem
* The primary platform for visualization in Python is [matplotlib](http://matplotlib.org/)
* Most other visualization platforms build on matplotlib
    * But see, e.g., [Bokeh](http://bokeh.pydata.org/en/latest/)


## Matplotlib
* Highly object-oriented (contrast with, e.g., R)
    * Easy to modify/customize plots created in different packages
* Documentation is comprehensive, but not very well organized
* Provides a basic set of high-level plots (barplot, scatter, etc.)
* Beyond that, API has a fairly steep learning curve

### Simple examples

In [ ]:
plt.scatter(data['min_weight'], data['height'], color='green', s=30)
plt.xlabel("Min. breed weight")
plt.ylabel("Min. breed height")
plt.title("Breed height vs. weight")

In [ ]:
# A heatmap of the crosstab we made earlier
plt.imshow(ctab[:10], interpolation='none', cmap='Reds')
plt.gca().grid('off')
plt.yticks(np.arange(10), ctab[:10].index)
plt.xticks(np.arange(7), ctab.columns, rotation=45)

## Customization in matplotlib
* matplotlib is infinitely customizable
* As in most modern plotting environments, you can do virtually anything
* You just have to be willing to spend enough time on it

### Matplotlib styles
<img src="https://raw.githubusercontent.com/rasbt/matplotlib-gallery/master/images/formatting_4.png">
https://twitter.com/rasbt/status/731205324187795457

In [ ]:
# A whole bunch of custom plotting code, just to generate
# a relatively simple result.
def plot_outcomes_by_time(data, unit='hour', min_unit=None, max_unit=None,
                          animal=None, panel=None, agg_func='count'):
    ''' Custom plotting functions that displays the number of
    outcomes of each type as a function of a unit of time and
    (optionally) any categorical variable.
    '''
    if animal is not None:
        data = data[data['animal'].isin(animal)]
    if panel is not None:
        if panel == 'year':
            panel_vars = [2014, 2015]
        else:
            panel_vars = data[panel].unique()
        n_panels = len(panel_vars)
        fig, axes = plt.subplots(1, n_panels, figsize=(4*n_panels, 4),
                                 sharex=True, sharey=True)
    else:
        axes = [plt.gca()]

    outcomes = ['Adoption', 'Transfer', 'Return to Owner', 'Euthanasia']
    dummies = pd.get_dummies(data['outcome'])
    data = pd.concat([data, dummies], axis=1)
    
    if min_unit is not None:
        data = data[data[unit] >= min_unit]
    if max_unit is not None:
        data = data[data[unit] <= max_unit]
    
    for oc in outcomes:
        groupers = [unit]
        if panel is not None:
            groupers.append(panel)
        line = data.query('outcome==@oc')\
            .groupby(groupers)[oc].agg(agg_func).reset_index()
        if panel is not None:
            for i, pan in enumerate(panel_vars):
                pan_line = line[line[panel]==pan]
                axes[i].plot(pan_line[unit], pan_line[oc], label=oc, lw=3)
                axes[i].set_title('{}: {}'.format(panel, pan), fontsize=18)
                axes[i].set_xlabel(unit, fontsize=18)
                axes[i].set_ylabel("No. of outcomes", fontsize=18)
    plt.legend(fontsize=16, loc='best')

In [ ]:
# Now we can flexibly call our custom plotter

# Plot outcome counts by hour, paneling by animal
plot_outcomes_by_time(data, animal=['Cat', 'Dog', 'Other'], unit='hour', panel='animal')
plt.gcf().set_size_inches((15, 5))

# # Plot outcome counts by day of week, only for dogs, paneling by year
# plot_outcomes_by_time(data, animal=['Dog'], unit='day', panel='year')
# plt.gcf().set_size_inches((10, 5))

### Alternative interfaces to matplotlib
* Most of the time you don't want to spend three hours making a nice barplot
* The appeal of packages like ggplot2 is power and elegance with speed
* A number of high-level Python plotting packages built on matplotlib exist
* You can always customize any plot after the fact

### ggplot for Python
* ggplot2 has been (mostly) [ported to Python](http://ggplot.yhathq.com/)
* Syntax is fairly similar
* Works for the most part, but not ready for prime time
    * For example...

In [ ]:
# This is the example on ggplot's front page
from ggplot import ggplot, aes, meat, geom_line, stat_smooth

ggplot(aes(x='date', y='beef'), data=meat) + \
    geom_line() + \
    stat_smooth(colour='blue', span=0.2)

## Plotting in seaborn
* Seaborn is a high-level plotting library based on matplotlib
    * seaborn : matplotlib :: ggplot2 :  base R
* Designed to produce attractive figures in very little code
* _Great_ [documentation](https://stanford.edu/~mwaskom/software/seaborn/index.html)
* Most seaborn plotting functions accept pandas DataFrames
* Complete customization of seaborn plots is possible using matplotlib

### Heatmaps

In [ ]:
# The same contingency table from earlier
ax = sns.heatmap(ctab[:12], annot=True, fmt='d', linewidths=.5, annot_kws={'size': 14})

# Customize the label size and figure size in base matplotlib.
# Notice we can access all axis and figure properties through
# the handle we saved when calling seaborn.
ax.tick_params(labelsize=14)
ax.figure.set_size_inches((7, 8))  # update the plot size

Let's visualize the correlation between all numeric variables in the dataset

In [ ]:
# Plot a heatmap of pairwise correlations between all continuous variables
r = data.query('animal=="Dog"').corr().round(2)
with sns.plotting_context("notebook", font_scale=1.1):
    ax = sns.heatmap(r, annot=True)
    ax.figure.set_size_inches((12, 9.5))

* Why do month and year show a strong negative correlation (-0.42)?
* Illustrates the need for caution
    * Most "interesting" findings turn out to have banal explanations!

### Pair plots
* Systematic inspection of pairwise relationships is very useful
* We've established that some breeds have very different outcomes
* How do outcomes relate to one another at the breed level?
    * Do some outcomes trade off against others?

In [ ]:
# Breed-level correlations between different outcomes and traits

# Restrict to breed with minimum 50 dogs and outcomes with min. 200 occurrences
_data = data.query('animal=="Dog"')
_data = _data.groupby('outcome').filter(lambda x: len(x) >= 200)
_data = _data.groupby('breed').filter(lambda x: len(x) >= 50)

# Dummy-code the outcomes as binary indicators and concatenate with original data
outcomes = pd.get_dummies(_data['outcome'])
_data = pd.concat([_data, outcomes], axis=1)

# Group by breed and calculate means
_data = _data.groupby(['group', 'breed']).mean().dropna().reset_index()
_data = _data[['Adoption', 'Euthanasia', 'Return to Owner', 'Transfer', 'group']]
dog_breed_means = _data  # Save for later

# The actual plotting code is just this
with sns.plotting_context("notebook", font_scale=1.3):  # Make text bigger
    sns.pairplot(_data, diag_kind='kde', kind='reg')
    
# Another way to look at the same data...
# with sns.plotting_context("notebook", font_scale=1.3):  # Make text bigger
#     sns.pairplot(_data, diag_kind='kde', hue='group')

* Let's explore how outcomes differ as a function of the variables in our dataset
* We'll start by visualizing the raw counts for different animals

In [ ]:
# Probably no point in keeping livestock and birds!
x_var = 'outcome'
panel_var = 'animal'
g = sns.factorplot(x=x_var, data=data, col=panel_var, kind='count')
g.set_xticklabels(rotation=45)

This is useful! Let's abstract it into a method. Then we can do the same thing (even more) quickly for other combinations.

In [ ]:
def aac_factorplot(data, x, facet, y=None, kind='count'):
    g = sns.factorplot(x=x, data=data, y=y, col=facet, kind=kind)
    g.set_xticklabels(rotation=45)
    
aac_factorplot(data, 'outcome', 'year')

#### Do temporal variables matter?
* We might expect day of the week to matter

In [ ]:
_data = data[data['animal'].isin(['Dog', 'Cat'])]
_data = _data.groupby('outcome').filter(lambda x: len(x) >= 500)
sns.factorplot(x='animal', data=_data, hue='outcome', col='day', kind='count') 

#### What about age?

In [ ]:
_data = data[data['animal'].isin(['Dog', 'Cat'])]
_data = _data.groupby('outcome').filter(lambda x: len(x) >= 500)
sns.factorplot(x='animal', y='age', data=_data, col='sterilized', hue='outcome', kind='bar')

#### Color?

In [ ]:
# Ideally, the colors should match the labels!
_data = data[data['animal'].isin(['Cat'])]
_data = _data.groupby('first_color').filter(lambda x: len(x) >= 200)
_data = _data.groupby('outcome').filter(lambda x: len(x) >= 500)
sns.factorplot(x='outcome', data=_data, hue='first_color', kind='count')
plt.gcf().set_size_inches((15, 4))

# Statistical analysis
* Python is historically not widely used for traditional statistics
    * It's improving fast, but will probably never catch up to R
* But it has decent support for the most common models
* Shines in:
    * Machine learning
    * Specific science domains--e.g., astrophysics, ecology, neuroimaging
    * More computationally-oriented domains--e.g., deep learning, Bayesian analysis

## Statistics functions in SciPy
* The SciPy package includes a well-developed [stats module](http://docs.scipy.org/doc/scipy/reference/stats.html)
* Caters more to computational scientists than end users
    * Interface is heavily object-oriented
    * You usually have to take the extra step yourself
    * Performance is very good, but interface is clunky

### Lots of descriptives and convenience functions

In [ ]:
# Descriptive statistics for age in months
print(stats.describe(data['age'].dropna()))

# Plenty of little convenience functions
age = 60
percentile = stats.percentileofscore(data['age'], age)
print("\nA {}-month old animal is at the {:.0f}th percentile.".format(age, percentile))

### Distributions a-plenty
* Huge number of statistical [distributions](http://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions)
* All of the standard distribution functions (pdf, cdf, random sampling, etc.) 

In [ ]:
# Sample 1000 observations from a bunch of distributions and plot
n = 10000
normal = stats.norm.rvs(size=n)
t = stats.t.rvs(df=10, size=n)
beta = stats.beta.rvs(a=2, b=5, scale=7, size=n)
ax = plt.figure(figsize=(6, 4)).gca()

for dist in [t, normal, beta]:
    sns.kdeplot(dist, ax=ax, lw=2)

In [ ]:
# Use CDF of normal to convert p-values to z-scores
print(stats.norm.ppf(0.001))

# t statistic to p-value
print(stats.t.cdf(1.46, df=22))

# Endpoints of interval containing 90% of distribution
print(stats.beta.interval(0.9, a=2, b=5))

### Basic tests
* You can do things like t-tests and ANOVAs with SciPy if you really want
* But there are more pleasant ways

In [ ]:
# Independent samples t-test: do cats and dogs processed at the center differ in age?

# Extract 
dog_age = data.query('animal=="Dog"')['age'].dropna()
cat_age = data.query('animal=="Cat"')['age'].dropna()

# ttest_ind returns a tuple of t and p values
t, p = stats.ttest_ind(dog_age, cat_age)

# Report means as well
dm, cm = dog_age.mean(), cat_age.mean()

# Print something for the user
print("Dogs (mean = {:.2f}) vs. Cats (mean = {:.2f}), t = {:.2f}, p={:.2g}".format(dm, cm, t, p))

## [Statsmodels](http://statsmodels.sourceforge.net/devel/index.html#)
* Python's answer to base R
* It's not a very good answer
* But it's getting there, and things [move quickly](https://github.com/statsmodels/statsmodels/issues)
* For common procedures, functionality is very similar to R

### What's in statsmodels
* Basic tests
* Linear models
* Diagnostics and specification tests
* Power calculations
* Multiple comparisons correction
* Time-series analysis
* Experimental support for many other things (e.g. mixed-effects models)

### Statsmodel examples
* Let's build a model to predict whether an animal is euthanized or not
* We'll use logistic regression with the native statsmodels API
* We'll start by predicting euthanasia from just age, sex, and sterilization

In [ ]:
# Drop missing values from analysis
_data = data[['outcome', 'age', 'sex', 'sterilized', 'mixed_color']].dropna()
print("Kept {} observations.".format(len(_data)))

# Add intercept manually for logistic model
_data['intercept'] = 1

# The dependent variable: whether the animal was euthanized
y = (_data['outcome'] == 'Euthanasia').astype(int)
_data.pop('outcome')

# Regression on continuous predictors is simple
model = sm.Logit(y, _data)

# Fit model and save result
result = model.fit()

# Print a user-friendly summary
result.summary()

### Adding categorical predictors
* Old animals are more likely to be euthanized
* Is this explained by differences in species, breed, etc.?
* We can add categorical predictors either by:
    * Constructing the dummy variables ourselves (e.g., with pandas)
    * Using the patsy module to do it in an R-like way

In [ ]:
# There are nearly 1,800 unique values for breed in our data.
# We can't get reasonable estimate for most of those; there
# are too few observations. Let's restrict analysis to
# common dog breeds--100 or more individuals.
_data = data.query('animal=="Dog"')
_data = _data.groupby('breed').filter(lambda x: len(x) >= 100)

# The NaNs in some columns might cause problems; let's replace them
cols = ['sex', 'sterilized']
_data[cols] = _data[cols].fillna('Unknown')

# Explicitly add the dependent variable
_data['y'] = (_data['outcome'] == 'Euthanasia').astype(int)

# We can easily set up the design matrix with an R-style formula
from patsy import dmatrices
y, X = dmatrices('y ~ age + breed + C(sex) + C(sterilized)', data=_data, return_type='dataframe')

results = sm.Logit(y, X).fit(missing=True)
results.summary()

In [ ]:
# The coefficients are in logit space; we can convert them to odds ratios
np.exp(results.params).sort_values()

# An easter egg: Probabilistic programming
* An up-and-coming paradigm for flexible statistical modeling
* Allows one to express virtually any type of model in code
* Doesn't require analytical derivation in order to fit the model
* Uses MCMC sampling to approximate the asymptotic answer
* Very powerful, but computationally intensive
* The state of the art is advancing rapidly
* Strong support in Python: [PyStan](https://pystan.readthedocs.io/en/latest/), [PyMC3](https://github.com/pymc-devs/pymc3), emcee, etc.

In [ ]:
# Installing pymc3 can take a little more work than other packages.
# Try running this cell, but don't freak out of it doesn't work.
!pip install Theano
!pip install git+https://github.com/pymc-devs/pymc3

In [ ]:
import pymc3 as pm

# Read in the data; keep only columns we need
_data = pd.read_csv('../data/preprocessed_data.csv')[['outcome', 'age', 'animal', 'sex', 'sterilized', 'breed']]

# Drop NAs, filter on breed and animal
_data = _data.dropna()
_data = _data.groupby('animal').filter(lambda x: len(x) >= 500).reset_index()
_data = _data.groupby('breed').filter(lambda x: len(x) >= 100).reset_index()

## Writing a PyMC3 model

In [ ]:
from sklearn.preprocessing import LabelEncoder

def invlogit(x):
    ''' Inverse logit '''
    return pm.exp(x) / (1 + pm.exp(x))

# Initialize PyMC3 model
model = pm.Model()

# Set dependent variable--whether or not animal is adopted
y = (_data['outcome']=='Adoption').astype(int).values

with model:
    
    # Initialize the predicted values to 0
    mu = 0.
    
    # Add fixed effects of sex, sterilization, and age
    b1 = pm.Normal('b_sex' ,mu=0, sd=10)
    b2 = pm.Normal('b_sterilized' ,mu=0, sd=10)
    b3 = pm.Normal('b_age' ,mu=0, sd=10)
    
    mu += b1*_data['sex'].values + b2*_data['sterilized'].values + b3*_data['age'].values
    
    # Add species intercepts
    b4 = pm.Normal('b_species', mu=0, sd=10, shape=2)
    animal_inds = LabelEncoder().fit_transform(_data['animal'].values)
    mu += b4[animal_inds]
    
# #     # Should really be modeled as a logistic regression, but for the
# #     # sake of speed and interpretable coefficients, we use the normal
    sigma = pm.HalfCauchy('sigma', 5)
    y_obs = pm.Normal('y_obs', mu=mu, sd=sigma, observed=y)
    
#     # uncomment to use logistic model
#     p = invlogit(mu)
#     y_obs = pm.Bernoulli('y_obs', p=p, observed=y)

### Sample!

In [ ]:
with model:
    trace = pm.sample(1000, step=pm.NUTS(), start=pm.find_MAP(), progressbar=True)

In [ ]:
p = pm.traceplot(trace[500:])

In [ ]:
pm.summary(trace[500:])